In [12]:
# Import all the required libraries

import numpy as np 
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from lightgbm import LGBMRegressor
import os
from time import time

In [2]:
# Loading the input excel files using pandas

train_df =  pd.read_excel('Data_Train.xlsx')
test_df=pd.read_excel('Test_set.xlsx')
big_df = train_df.append(test_df)

In [3]:
# Cleaning the data using the data preprocessing

big_df['Date'] = big_df['Date_of_Journey'].str.split('/').str[0].astype(int)
big_df['Month'] = big_df['Date_of_Journey'].str.split('/').str[1].astype(int)
big_df['Year'] = big_df['Date_of_Journey'].str.split('/').str[2].astype(int)

big_df=big_df.drop(['Date_of_Journey'], axis = 1)

In [4]:
big_df['Arrival_Time'] = big_df['Arrival_Time'] .str.split(' ').str[0]
big_df['Arrival_Hour'] = big_df['Arrival_Time'] .str.split(':').str[0].astype(int)
big_df['Arrival_Minute'] = big_df['Arrival_Time'] .str.split(':').str[1].astype(int)

big_df=big_df.drop(['Arrival_Time'], axis=1)

In [5]:
big_df['Dep_Hour'] = big_df['Dep_Time'] .str.split(':').str[0].astype(int)
big_df['Dep_Minute'] = big_df['Dep_Time'] .str.split(':').str[1].astype(int)

big_df=big_df.drop(['Dep_Time'], axis=1)

In [6]:
big_df['Total_Stops']=big_df['Total_Stops'].fillna('1 stop')
big_df['Total_Stops']=big_df['Total_Stops'].replace('non-stop','0 stop')
big_df['Stop'] = big_df['Total_Stops'].str.split(' ').str[0].astype(int)

big_df=big_df.drop(['Total_Stops'], axis=1)

In [7]:
big_df['Route_1'] = big_df['Route'] .str.split('→ ').str[0]
big_df['Route_2'] = big_df['Route'] .str.split('→ ').str[1]
big_df['Route_3'] = big_df['Route'] .str.split('→ ').str[2]
big_df['Route_4'] = big_df['Route'] .str.split('→ ').str[3]
big_df['Route_5'] = big_df['Route'] .str.split('→ ').str[4]

big_df['Route_1'].fillna("None",inplace = True)
big_df['Route_2'].fillna("None",inplace = True)
big_df['Route_3'].fillna("None",inplace = True)
big_df['Route_4'].fillna("None",inplace = True)
big_df['Route_5'].fillna("None",inplace = True)

In [8]:
big_df['Price'].fillna((big_df['Price'].mean()), inplace=True)

big_df=big_df.drop(['Route'], axis=1)
big_df=big_df.drop(['Duration'], axis=1)

In [9]:
#Enconding the categorical data using Label Encoder

from sklearn.preprocessing import LabelEncoder

lb_encode = LabelEncoder()
big_df["Additional_Info"] = lb_encode.fit_transform(big_df["Additional_Info"])
big_df["Airline"] = lb_encode.fit_transform(big_df["Airline"])
big_df["Destination"] = lb_encode.fit_transform(big_df["Destination"])
big_df["Source"] = lb_encode.fit_transform(big_df["Source"])
big_df['Route_1']= lb_encode.fit_transform(big_df["Route_1"])
big_df['Route_2']= lb_encode.fit_transform(big_df["Route_2"])
big_df['Route_3']= lb_encode.fit_transform(big_df["Route_3"])
big_df['Route_4']= lb_encode.fit_transform(big_df["Route_4"])
big_df['Route_5']= lb_encode.fit_transform(big_df["Route_5"])

In [10]:
#splitting the data to train and test data

df_train = big_df[0:10683]
df_test = big_df[10683:]
df_test = df_test.drop(['Price'], axis =1)

X = df_train.drop(['Price'], axis=1)
y = df_train.Price

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [17]:
df_test

,Airline,Source,Destination,Additional_Info,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
0,4,2,1,8,6,6,2019,4,25,17,30,1,3,7,6,12,4
1,3,3,0,8,12,5,2019,10,20,6,20,1,2,33,3,12,4
2,4,2,1,5,21,5,2019,19,0,19,15,1,3,7,6,12,4
3,6,2,1,8,21,5,2019,21,0,8,0,1,3,7,6,12,4
4,0,0,2,8,24,6,2019,2,45,23,55,0,0,13,24,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,8,6,6,2019,20,25,20,30,1,2,14,3,12,4
2667,3,3,0,8,27,3,2019,16,55,14,20,0,2,5,24,12,4
2668,4,2,1,8,6,3,2019,4,25,21,50,1,3,7,6,12,4
2669,1,2,1,8,6,3,2019,19,15,4,0,1,3,7,6,12,4


In [32]:
lm = LinearRegression()
kfolds = KFold(n_splits=50,shuffle=True, random_state=100)
def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, 
                                   scoring="neg_mean_squared_error", 
                                   cv = kfolds))
    return(rmse)


t1 = time()
benchmark_model = make_pipeline(StandardScaler(),lm).fit(X_train, y_train)
cv_rmse(benchmark_model).mean()

print(f'The rmse value is: {cv_rmse(ridge_model).mean()}')
t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 3238.090389193024
Time taken to run the model 124.92794418334961 seconds


In [25]:
t1 = time()
alphas_ridge = [20, 25, 27.5, 28.5, 29, 29.5, 30, 32.5, 35, 37.5, 40]

ridge = make_pipeline(StandardScaler(),
                            RidgeCV(alphas = alphas_ridge,
                                    cv=kfolds))
ridge_model =ridge.fit(X_train, y_train)

print(f'The rmse value is: {cv_rmse(ridge_model).mean()}')
t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 3238.090389193024
Time taken to run the model 120.80717039108276 seconds


In [24]:
t1 = time()
alphas_lasso = [0.00005, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
           0.0006, 0.0007, 0.0008]

lasso = make_pipeline(StandardScaler(),
                             LassoCV(max_iter=1e7,
                                    alphas = alphas_lasso,
                                    random_state = 42))

lasso_model = lasso.fit(X_train, y_train)

print(f'The rmse value is: {cv_rmse(lasso_model).mean()}')
t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 3238.229061371743
Time taken to run the model 5.903466463088989 seconds


In [21]:
t1 = time()
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]

elasticnet = make_pipeline(StandardScaler(), 
                           ElasticNetCV(max_iter=10000, alphas=e_alphas, 
                                        cv=kfolds))
elasticnet_model = elasticnet.fit(X_train, y_train)
print(f'The rmse value is: {cv_rmse(elasticnet_model).mean()}')

t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 3238.20048894392
Time taken to run the model 22.84486675262451 seconds


In [29]:
from xgboost import XGBRegressor

t1 = time()
xgb = XGBRegressor(learning_rate =0.1, n_estimators=200, max_depth=10)


xgb_model = make_pipeline(StandardScaler(),xgb).fit(X_train, y_train)

print(f'The rmse value is: {cv_rmse(xgb_model).mean()}')

t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 1506.512774540475
Time taken to run the model 82.72026991844177 seconds


In [30]:
from lightgbm import LGBMRegressor

t1 = time()
lgbm = LGBMRegressor(objective='regression',num_leaves=31,
                              learning_rate=0.1, n_estimators=200)
lgbm_model = make_pipeline(StandardScaler(),lgbm).fit(X=X_train, y=y_train)

print(f'The rmse value is: {cv_rmse(lgbm_model).mean()}')

t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds') 

The rmse value is: 1471.8798061823636
Time taken to run the model 15.85589599609375 seconds


In [31]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.pipeline import make_pipeline

#stack

t1 = time()
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, 
                                            xgb, lgbm), 
                               meta_regressor=xgb)

stack_gen_model = make_pipeline(StandardScaler(),stack_gen).fit(X_train, y_train)


print(f'The rmse value is: {cv_rmse(stack_gen_model).mean()}')

t2 = time()
print(f'Time taken to run the model {t2 - t1} seconds')

The rmse value is: 1619.1892907867307
Time taken to run the model 1326.2168107032776 seconds


In [33]:
benchmark_preds = benchmark_model.predict(X_test)
ridge_preds = ridge_model.predict(X_test)
lasso_preds = lasso_model.predict(X_test)
elastic_preds = elasticnet_model.predict(X_test)

xgb_preds = xgb_model.predict(X_test)
lgbm_preds = lgbm_model.predict(X_test)
stack_preds = stack_gen_model.predict(X_test)

In [55]:
from sklearn.metrics import mean_squared_error, r2_score

for preds in [benchmark_preds, ridge_preds, lasso_preds, elastic_preds, xgb_preds, lgbm_preds,stack_preds]:
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2score = r2_score(y_test, preds)
    #evaluvation = np.sqrt(np.square(np.log10(preds+1) - np.log10(y_test+1)).mean())
    print(f'rmse : {rmse} and {r2score}')

rmse : 3228.7471774901055 and 0.4834156699537322
rmse : 3228.756501067724 and 0.4834126864924778
rmse : 3228.7470862652563 and 0.4834156991448235
rmse : 3228.736671916637 and 0.48341903163273914
rmse : 1549.2567557055822 and 0.8810621875799216
rmse : 1400.585423269352 and 0.9027941609453528
rmse : 1521.84423812757 and 0.885233917372131


In [50]:
pred_price = np.concatenate((y_test.values.reshape(3205,1), benchmark_preds.reshape(3205,1), ridge_preds.reshape(3205,1),
                     lasso_preds.reshape(3205,1), elastic_preds.reshape(3205,1), 
                             xgb_preds.reshape(3205,1), lgbm_preds.reshape(3205,1), 
                     stack_preds.reshape(3205,1)),1)
pred_price[:8]

array([[ 3597.        ,  4036.31867119,  4089.29241324,  4036.32439455,
         4040.19699654,  3601.89428711,  3745.1572884 ,  3677.95678711],
       [ 3383.        ,  6042.10890558,  6057.0739608 ,  6042.10755275,
         6043.12292331,  3771.77368164,  3729.24086985,  3863.20727539],
       [ 2050.        ,  3458.56415449,  3494.60425291,  3458.57116106,
         3461.17984662,  2477.27905273,  2665.42023076,  2579.97412109],
       [ 4423.        ,  4602.81080011,  4610.79179376,  4602.80942448,
         4603.29004764,  4014.59399414,  4316.18568524,  3932.59106445],
       [ 3597.        ,  6384.11136002,  6438.1992012 ,  6384.11751368,
         6388.17176746,  3743.88793945,  4185.57874298,  3902.58544922],
       [10844.        ,  9765.20462016,  9770.01237205,  9765.20721513,
         9765.62333024,  9595.70019531, 10107.89477426,  9992.90625   ],
       [13227.        , 12810.62778426, 12817.13566067, 12810.62999611,
        12811.24124555, 13184.546875  , 14249.09468452, 14

In [51]:
i =1
for model in [benchmark_model, ridge_model, lasso_model, elasticnet_model, xgb_model, lgbm_model,stack_gen_model]:
    test_df_excel = test_df.copy()
    preds_1 = model.predict(df_test)
    test_df_excel['Price'] = preds_1
    if os.path.exists('flight_fare.xlsx'):
        with pd.ExcelWriter('flight_fare.xlsx', engine="openpyxl", mode = 'a') as writer:
            test_df_excel.to_excel(writer, sheet_name = 'flight_price_' + str(i))
            i+=1
    else:
        test_df_excel.to_excel('flight_fare.xlsx', sheet_name = 'flight_price_' + str(i))
        i+=1